Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [2]:
PYTHONOPTIMIZE=1

In [3]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import heapq
import functools
import numpy as np
from icecream import ic

## Instance generator for the N-Puzzle problem

In [4]:
PUZZLE_DIM = 3
PUZZLE_SIZE = PUZZLE_DIM * PUZZLE_DIM
action = namedtuple('Action', ['pos1', 'pos2'])

In [5]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def counter(fn):
    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)
    helper.calls = 0
    return helper

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def is_solvable(puzzle, goal):
    puzzle_flat = puzzle.flatten()
    goal_flat = goal.flatten()

    goal_positions = {value: idx for idx, value in enumerate(goal_flat)}
    puzzle_mapped = [goal_positions[val] for val in puzzle_flat if val != 0]

    inversions = 0
    for i in range(len(puzzle_mapped)):
        for j in range(i + 1, len(puzzle_mapped)):
            if puzzle_mapped[i] > puzzle_mapped[j]:
                inversions += 1

    blank_row_from_bottom = PUZZLE_DIM - np.where(puzzle == 0)[0][0]
    # ic(blank_row_from_bottom)

    if PUZZLE_DIM % 2 == 1:  # Odd-sized puzzle
        return inversions % 2 == 0
    else:  # Even-sized puzzle
        return (blank_row_from_bottom % 2 == 0) == (inversions % 2 == 0)

Generate iteratively the solution as goal_state

In [6]:
def generate_solution(n):
    matrix = np.zeros((n, n), dtype=int)
    num = 1
    stop = n * n
    top, bottom, left, right = 0, n - 1, 0, n - 1

    # i have to fill until the matrix collapses into a single element ( top = bottom and left = right )
    while top <= bottom and left <= right:
        if num == stop:
            break
        for i in range(left, right + 1):
            matrix[top, i] = num
            num += 1
        top += 1
        if num == stop:
            break
        for i in range(top, bottom + 1):
            matrix[i, right] = num
            num += 1
        right -= 1
        # in case of 2*2 matrix, we don't need to fill the bottom row, so let's check
        if top <= bottom and num != stop:
            for i in range(right, left - 1, -1):
                matrix[bottom, i] = num
                num += 1
            bottom -= 1
        else:
            break
        # same here
        if left <= right != stop:
            for i in range(bottom, top - 1, -1):
                matrix[i, left] = num
                num += 1
            left += 1
        else:
            break
    return matrix

Randomizing the initial matrix

In [7]:
RANDOMIZE_STEPS = (PUZZLE_DIM * PUZZLE_DIM-1) * 1000
goal_state = generate_solution(PUZZLE_DIM)

# state = np.array([i for i in range(1, PUZZLE_SIZE)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
# for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
#     state = do_action(state, choice(available_actions(state)))

# Randomizing until i found a solvable puzzle
solvable = False
while not solvable:
    # instead of generating a random state, i will generate a solution and randomize it
    if PUZZLE_DIM % 2 != 0:
        state = goal_state.copy()
    else:
        state = np.array([i for i in range(1, PUZZLE_SIZE)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    for r in range(RANDOMIZE_STEPS):
        state = do_action(state, choice(available_actions(state)))
    solvable = is_solvable(state, goal_state)
ic(state, solvable)
None

ic| state: array([[1, 3, 7],
                  [4, 6, 5],
                  [0, 2, 8]])
    solvable: True


## A* search

### Heuristic distance

- Manhattan distance
- Linear conflict distance ( found online when i was reading about manhattan distance )

In [8]:
def manhattan_distance(curr, goal):
    distance = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            if curr[i][j] != 0:
                goal_i, goal_j = np.where(goal == curr[i][j])
                distance += abs(goal_i - i) + abs(goal_j - j)
    return distance

def linear_conflict_distance(curr, goal):
    distance = 0
    linear_conflict = 0

    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            if curr[i][j] != 0:
                # Compute Manhattan distance
                goal_i, goal_j = np.where(goal == curr[i][j])
                distance += abs(goal_i - i) + abs(goal_j - j)
                # Check for linear conflicts in the row
                if goal_i == i:
                    for k in range(j + 1, PUZZLE_DIM):  # Compare with other tiles in the row
                        if curr[i][k] != 0: 
                            goal_k_i, goal_k_j = np.where(goal == curr[i][k])
                            if goal_k_i == i and goal_j > goal_k_j:
                                linear_conflict += 2
                # Check for linear conflicts in the column
                if goal_j == j:
                    for k in range(i + 1, PUZZLE_DIM):  # Compare with other tiles in the column
                        if curr[k][j] != 0:
                            goal_k_i, goal_k_j = np.where(goal == curr[k][j])
                            if goal_k_j == j and goal_i > goal_k_i:
                                linear_conflict += 2

    return distance + linear_conflict

def to_tuple(state):
    return tuple(map(tuple, state))

def to_matrix(tuple):
    return np.array(tuple).reshape((PUZZLE_DIM, PUZZLE_DIM))

### A* algorithm

In [9]:
def a_star_algo(start, gol, heuristic):
    open_list = [] # all the nodes that we have to visit
    close_list = set() # all the nodes that we have visited
    start_tuple = to_tuple(start)
    heapq.heappush(open_list, (0, start_tuple, 0)) # (f_score, state, 0)
    came_from = {}
    g_score = {start_tuple: 0} # cost for each node stored in a dictionary
    goal_tuple = to_tuple(gol)

    while open_list: 

        _, current, cost = heapq.heappop(open_list)

        # terminate if we reach the goal state and build the path by backtracking the came_from dictionary
        if current == goal_tuple:
            ic(len(open_list), len(close_list))
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start_tuple)
            return (1, path[::-1])
        current_state = to_matrix(current)

        if current in close_list and g_score[current] < cost:
            continue
        close_list.add(current)

        # fix this ?
        for act in available_actions(current_state):
            new_state = do_action(current_state, act)
            new_state_tuple = to_tuple(new_state)
            new_cost = cost + 1

            if new_state_tuple not in g_score or new_cost < g_score[new_state_tuple]:
                g_score[new_state_tuple] = new_cost
                f_score = new_cost + heuristic(new_state, gol)
                heapq.heappush(open_list, (f_score, new_state_tuple, new_cost))
                came_from[new_state_tuple] = current

    return (0, came_from)

## Solver

In [8]:
# A sample test case ( solved in 26 steps )
sample = np.array([1, 2, 7, 4, 5, 6, 3, 0, 8]).reshape((3, 3))
sample2 = np.array([7, 6, 3, 1, 4, 8, 0, 2, 5]).reshape((3, 3))
# making sample2 solvable by swapping 1 and 4
sample3 = np.array([7, 6, 3, 4, 1, 8, 2, 0, 5]).reshape((3, 3))
if not is_solvable(sample, goal_state):
    print("Sample is not solvable")
if not is_solvable(sample2, goal_state):
    print("Sample2 is not solvable")
if not is_solvable(sample3, goal_state):
    print("Sample2 is not solvable even after swapping 1 and 4")
if not is_solvable(state, goal_state):
    print("State is not solvable")
samples = [sample, sample2, sample3, state]

Sample2 is not solvable


Comparison between manatthan distance and linear conflict speed

In [10]:
solution_manhattan = a_star_algo(state, goal_state, manhattan_distance)
if solution_manhattan[0] == 1 :
    path = solution_manhattan[1]
    print(f"Manhattan distance: {len(path)}")
else:
    print('No solution')

solution_linear_conflict = a_star_algo(state, goal_state, linear_conflict_distance)
if solution_linear_conflict[0] == 1 :
    path = solution_linear_conflict[1]
    print(f"Linear conflict distance: {len(path)}")
else:
    print('No solution')

ic| len(open_list): 172, len(close_list): 275
ic| len(open_list): 126, len(close_list): 202


Manhattan distance: 23
Linear conflict distance: 23


Linear conflict is faster as it is more efficient, so i'll continue using this approach. Here an example of a 4x4

In [35]:
solution_linear_conflict = a_star_algo(state, goal_state, linear_conflict_distance)
if solution_linear_conflict[0] == 1 :
    path = solution_linear_conflict[1]
    print(f"Solved\n{state}\nwith linear conflict distance: {len(path)}")
else:
    print(f'{state} has no solution')

ic| len(open_list): 102535, len(close_list): 117444


Solved
[[11 14  0  3]
 [ 4  5  1 10]
 [13  8 12  2]
 [15  7  9  6]]
with linear conflict distance: 50


Solving state and samples

In [195]:
for sol in samples:
    solution_linear_conflict = a_star_algo(sol, goal_state, linear_conflict_distance)
    if solution_linear_conflict[0] == 1 :
        path = solution_linear_conflict[1]
        print(f"Solved\n{sol}\nwith linear conflict distance: {len(path)}")
    else:
        print(f'{sol} has no solution')

ic| len(open_list): 403, len(close_list): 725


Solved
[[1 2 7]
 [4 5 6]
 [3 0 8]]
with linear conflict distance: 26


ic| len(open_list): 437, len(close_list):

[[7 6 3]
 [1 4 8]
 [0 2 5]] has no solution


 754
ic| len(open_list): 35, len(close_list): 52


Solved
[[7 6 3]
 [4 1 8]
 [2 0 5]]
with linear conflict distance: 26
Solved
[[0 7 4]
 [3 8 5]
 [1 2 6]]
with linear conflict distance: 19


Trying with 10 random puzzles

In [ ]:
RANDOMIZE_STEPS = 100
for i in tqdm(range(10), desc='Trying 10 random puzzles'):

    state = np.array([i for i in range(1, PUZZLE_SIZE)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    for r in range(RANDOMIZE_STEPS):
        state = do_action(state, choice(available_actions(state)))

    solution_linear_conflict = a_star_algo(state, goal_state, linear_conflict_distance)
    if solution_linear_conflict[0] == 1 :
        path = solution_linear_conflict[1]
        print(f"Linear conflict distance for puzzle {i}: {len(path)}")
    else:
        print(f'No solution for puzzle {i}')